<img src="../assets/ittc_logo_full.png" height=150 />

# Practical 6: Regression

In this workshop you will:

* Perform linear regression on existing linear data
* Analyze residuals and determine goodness of fit
* Transform non-inear into a linear form
* Make predictions based on these models

## 1. Getting started

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

concrete = pd.read_csv("../data/concrete.csv")
concrete = concrete.query("CompressiveStrength < 100")  # Filter the outlier

## 2. Concrete: a univariable model

As a first exercise, let't attempt to see if the CompressiveStrength of concrete can be modelled linearly based on the Cement content.

First let's begin by plotting CompressiveStrength as a function of Cement. On a first pass, does the relationship look approximately linear?

In [ ]:
# plt.scatter(concrete["Cement"], concrete["CompressiveStrength"])

### 2.1 Building the model

Now using the formula notation from statsmodels, lets model this degradation using an Ordinary Least Squares model. This process looks like:

`model = smf.ols(formula="y ~ x", data=df).fit()`

But you will need to replace the data and the names of independent and dependent variables: `Cement` and `CompressiveStrength`.

In [ ]:
# model = smf.ols(formula="...", data=concrete).fit()

The regression object (`model`) contains a number of useful properties and methods:

* `model.params` is a list of the fitted parameters, in this case ordered as `[slope, intercept]`
* `model.resid` is a list of residuals, i.e. $y - \hat{y}$
* `model.summary()` or `model.summary(slim=True)` returns a summary of the fit, its parameters and the goodness of fit tests
* `model.predict()` returns the model's predicted values for each of the original independent variables (e.g. time)


In [ ]:
# Experiment with some of these values


### 2.2 Assessing goodness of fit

Let's explore some of these to determine whether our model fits the data well.

First, let's assess the model using its summary field. Does the $R^2$ value indicate a good fit?

In [ ]:
# model.summary(slim=True)

Next let's plot our model:

1. Compare model with the underlying values
2. Compare the variance of residuals
3. Plot the QQ plot of the residuals to determine if they are normally distributed

In [ ]:
def make_comparison_plot(dataframe, xvar, yvar, model_fit_obj):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

    ax1.scatter(concrete[xvar], concrete[yvar])
    ax1.plot(concrete[xvar], model_fit_obj.predict(), color="red")
    ax1.set_xlabel(f"{xvar} unit")
    ax1.set_ylabel(yvar)
    ax1.set_title(f"{xvar} Model vs Values")

    ax2.scatter(concrete[xvar], model_fit_obj.resid)
    ax2.grid()
    ax2.set_ylabel("Residual")
    ax2.set_xlabel("{xvar} unit")
    ax2.set_title("Residuals")

    sm.qqplot(model_fit_obj.resid_pearson, line="45", ax=ax3)
    ax3.set_title("QQ Plot of residuals")
    plt.suptitle(f"Analysis of Linear fit of {yvar} vs {xvar} units")
    plt.show()


make_comparison_plot(concrete, xvar="Cement", yvar="CompressiveStrength", model_fit_obj=model)

Does the model seem reasonable? Remember to ask yourself:

1. Are the residuals normally distributed about the regression line?
2. Is this residual variation constant with respect to time?
3. Does the original data appear linear?
4. Are the observations independent?

## 3. Concrete: a multivariable model

The previous model based on the single variable `Cement` did not sufficiently meet our conditions.

What if we try a more complex model based on all of the parameters?

### 3.1 Building the model

Consider just 3 independent variables, `a`, `b`, `c`. Let's build a model that includes:

* Each variable
* Each comination of variables ("coupled terms")

Or more explicitly: `y ~ a + b + c + a b + b c + a c`

StatsModels formulas let us do this a little more succinctly: `y ~ (a + b + c)**2`

Your task: fit a model model to each of the parameters in the `concrete` data DataFrame, as well as each combination of parameters.

You can use the function we just made (`make_comparison_plot()`) to help visualise your result

In [ ]:
# model = smf.ols(formula="...", data=concrete).fit()

# make_comparison_plot(concrete, xvar="Cement", yvar="CompressiveStrength", model_fit_obj=model)

In [ ]:
# Another fit

In [ ]:
# another fit

In [ ]:
# Coupled fit! use "y ~ (a + b + c)**2", but replace y, a, b, c, with the variables in our dataframe

### 3.2 Assessing the goodness of fit

Now have a look at the output of `model.summary(slim=True)`.

* What does this tell you about the model?
* Are there parameters are not statistically strong predictors of the CompressiveStrength?

In [ ]:
# model.summary(slim=True)

It is not easy to plot the model (since there are 8 input variables) but we can plot the residuals as well as the QQ plot to check for deviations from a normal distribution.

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
# Plot the residuals and QQ plot on axes 1 and 2.

Those residuals are quite large (~20) compared to the underlying values of the CompressiveStrength.

Plot the actual value as a fraction of the predicted value using a BoxPlot. Do you think the model has good predictive power?

In [ ]:
# plt.boxplot(...);

## 3. Transformations

Often a dataset is not linear and requires some transformation to make it so. In this final section we consider how to construct models from data that has been linearised under transformation.

### 3.1 The model: ore fraction based on light absorbance

Consider the following dataset that measures:

* Ore fraction: the percentage of a particular mineral ore in particular sample
* Aborbance: the absorbance of light at a particular wavelength [arbitrary units]

**Our goal:** use a regression model to predict the amount of ore in a sample based on its absorbance.

In [ ]:
ore = pd.read_csv("../data/ore.csv")
ore.sort_values("Absorbance", inplace=True)
display(ore)

First, plot the ore fraction (`Amount`) as a function of the `Absorbance`. Does it look linear?

In [ ]:
plt.scatter(ore["Amount"], ore["Absorbance"])

### 3.2 Transforming the data

The data is clearly not linear. It is our hope that there exists a simple transformation that make this data linear.

![](images/BulgingRule.png)

Work you way through each of the following transformations applied to the `Amount`. Which of these best linearises this data?

* `np.sqrt(x)`
* `np.log(x)`
* `np.power(x, 1/3)` (cubed root)
* `np.power(x, 2)` (squared)
* `np.power(x, 3)` (cubed)


In [ ]:
# plt.scatter(ore["Absorbance"], ...) vs Amount

In [ ]:
# Another plt.scatter()

In [ ]:
# Another plot

Once you have found the best transformation, we can create a model on the linearised data:

Use a formula like: `np.power(y, 2) ~ x`

In [ ]:
# model = smf.ols(formula="... ~ Absorbance", data=ore).fit()
# model.summary(slim=True)

### 3.3 Assessing goodness of fit

How does well does this model work for our transformed data set?

Let's plot the model, residuals, and QQ plot:

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))

ax1.scatter(ore["Absorbance"], np.power(ore["Amount"], 1 / 3))
ax1.plot(ore["Absorbance"], model.predict(), color="red")

ax2.scatter(ore["Absorbance"], model.resid)
ax2.grid(True)

sm.qqplot(model.resid_pearson, ax=ax3, line="45")

### 3.4 Confidence interval

Remember that our model is subject to a certain amount of uncertainty which is captured by the **confidence** and **prediction** intervals.

Use the `model.get_prediction().summary_frame(alpha=0.05)` to get a DataFrame with the lower and upper values for the model's prediction interval (`obs_ci_lower` and `obs_ci_higher`, respectively).

In [ ]:
# prediction = ...
# display(prediction)

### 3.5 Inverting the transformation


To actually make predictions of the ore fraction, we need to reverse the transformation. i.e. appply the **inverse operation**.

For example, if we had chosen `np.power(x, 2)` as our transformation, the model prediction would be the square root. e.g. `np.sqrt(prediction["mean"])`

Find the inverse transformation and plot the predicted values of `Amount` **and** the lower and upper confidence interval:

In [ ]:
# # Apply the inverse transformation to each of these columns:
# modelvalues = prediction["mean"]
# ci_low = prediction["obs_ci_lower"]
# ci_high = prediction["obs_ci_upper"]

fig, ax = plt.subplots()
ax.fill_between(ore["Absorbance"], ci_low, ci_high, color="red", alpha=0.2)
ax.set_xlabel("Absorbance")
ax.set_ylabel("Amount")
ax.scatter(ore["Absorbance"], ore["Amount"])
ax.plot(ore["Absorbance"], modelvalues, color="red")